# Code to get phase at any desired MJD for multiple frequencies
This code was written to run the polyco mode on tempo interatively for multiple frequencies in the process two awk commands were written

1. To run tempo polyco with desired paramters without going to the command line 
2. To capture the important parameters from the output table in the output polyco file 

Before running this code please see to it that these awk files are executable and are present in pwd.Also the par file for(J0835-4510.par) for the pulsar should be available , currently this has not been tailored for multiple pulsars work is underway. 

In [1]:
import subprocess
import pandas as pd
import numpy as np

# parameters (change as needed) The code should work for sweeping multiple paramters in polyco ideally made to test polyco while avoiding to run it multiple times
mjd_start_values = [56363] #UTC
mjd_end_values = [56364] #UTC
nspan_values = [60] # in min 
ncoeffs_values = [3] #this is currently fixed do not change this 
maxha_values = [12]
telescope_code_values = ['pks']
frequencies = [320,1368.602051] 

# generate input files for different parameter combinations
with open("output_file.txt", 'w') as file:
    for frequency in frequencies:
        for mjd_start in mjd_start_values:
            for mjd_end in mjd_end_values:
                for nspan in nspan_values:
                    for ncoeffs in ncoeffs_values:
                        for maxha in maxha_values:
                            for telescope_code in telescope_code_values: 
                                    output_file = f"tempo2_input_{mjd_start}_{mjd_end}_{nspan}_{ncoeffs}_{maxha}_{telescope_code}_{frequency}.txt"
                                    parameter_values = [mjd_start, mjd_end, nspan, ncoeffs, maxha, telescope_code, frequency]
                                    line = " ".join(map(str, parameter_values)) + "\n"
                                    file.write(line)

#AWK commands

#To capture the paramaters from the parameter sweep file and run tempo2 polyco on it 
with open("tempo_param_capture.awk", 'r') as awk_file:
    awk_command = awk_file.read()
subprocess.run(f'awk \'{awk_command}\' output_file.txt | sh', shell=True)



This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 
This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 


CompletedProcess(args='awk \'# awk_command.awk\n\n{print "tempo2 -f J0835-4510.par -polyco \'\\\'\'" $0 "\'\\\'\' -polyco_file \'\\\'\'" $7 "\'\\\'\'"}\n\n\' output_file.txt | sh', returncode=0)

1. This will run the tempo2 polyco commands for all the frequencies and name the files accordingly.  
2. Check if the polyco outputs are generated in the pwd 


In [2]:
#The output from polyco is stored as {frequency of observation}_polyco_new.dat 
!ls

1368.602051newpolyco.dat   fsweep_extract_coeff.awk
1368.602051polyco_new.dat  J0835-4510.par
1368.602051polyco.tim	   J0835-4510.tim
320newpolyco.dat	   output_file.txt
320polyco_new.dat	   param_sweep_tempo2_awk.ipynb
320polyco.tim		   tempo_param_capture.awk


Next step is to check the awk file fsweep_extract_coeff.awk has the correct frequencies listed 

In [4]:
!cat fsweep_extract_coeff.awk


#!/usr/bin/awk -f

BEGIN {
    # Define an array of frequencies
    frequencies_str = "320,1368.602051";  # Add more frequencies if needed
    split(frequencies_str, frequencies);

    # Loop through the frequencies and process each one
    for (i = 1; i <= length(frequencies); i++) {
        # Set the output file for coefficients based on the current frequency
        coeff_file = "coeff_data_" frequencies[i] ".dat";

        # Print header for the coefficient file
        print "PSRNAME epoch f0 RPHASE COEFF1 COEFF2 COEFF3" > coeff_file;
    }
}

(NR-1) % 3 == 0 {
    epoch = $4;
    PSRNAME = $1
} 

NR % 3 == 2 {
    RPHASE = $1;
    f0 = $2
}

NR % 3 == 0 {
    COEFF1 = $1; COEFF2 = $2; COEFF3 = $3

    # Loop through the frequencies and print data to the corresponding file
    for (i = 1; i <= length(frequencies); i++) {
        coeff_file = "coeff_data_" frequencies[i] ".dat";
        print PSRNAME, epoch,f0,RPHASE, COEFF1, COEFF2, COEFF3 >> coeff_file;
    }
}



In [5]:
#From the polyco formula to get the phase 
def calculate_phase_and_frequency(mjd, tmid, rphase, coeff1, coeff2, coeff3,f0):
    # Calculate the time difference in minutes
    dt = abs((mjd - tmid) * 1440)

    # Calculate phase
    phase = rphase + dt * 60 * f0 + coeff1 + dt * coeff2 + dt**2 * coeff3
    
    # Calculate frequency in Hz
    freq = f0+ (1 / 60) * (coeff2 + 2 * dt * coeff3)

    return phase, freq

In [ ]:
polyco_table=[] #initiate dummy df 
#freq_sweep = [320,322,324,326,330,332,340,380,400]
mjd_input = 56364  # Replace with the desired MJD value
f0 = 11.188233045354 #from the par file 
output_pred = [] #array to store the desired output values 


for frequ in range(len(frequencies)):
    
    #To extract the coefficients and required constants from output of tempo2 to calculate phase and frequency 
    with open("fsweep_extract_coeff.awk", 'r') as awk_file: #please check the fsweep_extract_coeff.awk file if it has same array of frequencies
        awk_command = awk_file.read()
        command = f'awk \'{awk_command}\' {frequencies[frequ]}polyco_new.dat'
        subprocess.run(command, shell=True, check=True)
    
    file_path = f'coeff_data_{frequencies[frequ]}.dat'
    polyco_table = pd.read_csv(file_path, delim_whitespace=True, comment='#', names=['Pulsar', 'TMID', 'F0','RPHASE', 'COEFF1', 'COEFF2', 'COEFF3'],skiprows=1)
    
    for i in range(len(polyco_table)):
        phase,freq= calculate_phase_and_frequency(mjd_input,polyco_table['TMID'][i],polyco_table['RPHASE'][i],polyco_table['COEFF1'][i],polyco_table['COEFF2'][i],polyco_table['COEFF3'][i],f0)
        #print(phase)
        output = {'Pulsar': polyco_table['Pulsar'][i],
                  'Freq': frequencies[frequ],
                  'TMID': polyco_table['TMID'][i],
                  'MJD': mjd_input,
                  'Phase': phase,
                  'RPHASE': polyco_table['RPHASE'][i],
                  'Frequency': freq}

        output_pred.append(output)
        print(f"Pulsar: {output['Pulsar']}, Freq = {output['Freq']}, MJD: {output['MJD']}, Phase: {output['Phase']}, Frequency: {output['Frequency']} Hz")

Check if the code created the files containg the required extracted paramters from the polyco table 

In [14]:
!ls #check for files coeff_data_{frequency}.dat

1368.602051newpolyco.dat	coeff_data_320.dat
1368.602051polyco_new.dat	fsweep_extract_coeff.awk
1368.602051polyco.tim		J0835-4510.par
320newpolyco.dat		J0835-4510.tim
320polyco_new.dat		output_file.txt
320polyco.tim			param_sweep_tempo2_awk.ipynb
coeff_data_1368.602051.dat	tempo_param_capture.awk
coeff_data_320,1368.602051.dat


In [ ]:
#check the polyco table 
polyco_table

In [29]:
output_pred

[{'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.0,
  'MJD': 56364,
  'Phase': 3316.8419866298755,
  'RPHASE': -963334.937563,
  'Frequency': 11.188125913415435},
 {'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.0416666666,
  'MJD': 56364,
  'Phase': 3316.643112507847,
  'RPHASE': -923057.872029,
  'Frequency': 11.188121199589455},
 {'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.0833333333,
  'MJD': 56364,
  'Phase': 3316.3075015344716,
  'RPHASE': -882780.799172,
  'Frequency': 11.188112901996309},
 {'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.125,
  'MJD': 56364,
  'Phase': 3315.882076185454,
  'RPHASE': -842503.720412,
  'Frequency': 11.188101900272914},
 {'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.1666666666,
  'MJD': 56364,
  'Phase': 3315.4136371602904,
  'RPHASE': -802226.637684,
  'Frequency': 11.188089199304216},
 {'Pulsar': '0835-4510',
  'Freq': 320,
  'TMID': 56363.2083333333,
  'MJD': 56364,
  'Phase': 3314.944920836417,
  'RPHASE

In [30]:
#This is the output file 
output_pred[1]['Pulsar']

'0835-4510'

In [31]:
output_df = pd.DataFrame(output_pred)
output_df['Freq']

0      320.000000
1      320.000000
2      320.000000
3      320.000000
4      320.000000
         ...     
91    1368.602051
92    1368.602051
93    1368.602051
94    1368.602051
95    1368.602051
Name: Freq, Length: 96, dtype: float64